## 단층 퍼셉트론 구현

In [ ]:
# detach / torch.no_grad() 차이점 : https://saturncloud.io/blog/difference-between-detach-and-with-torchnograd-in-pytorch/

In [2]:
import torch
import torch.nn as nn

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [4]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

linear = nn.Linear(2, 1, bias=True)
sigmoid = nn.Sigmoid()
model = nn.Sequential(linear, sigmoid).to(device)

In [5]:
# 비용 함수와 옵티마이저 정의
criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)

In [6]:
#1,001번의 에포크 수행. 0번 에포크부터 1,000번 에포크까지.
for step in range(1001): 
    optimizer.zero_grad()
    hypothesis = model(X)

    # 비용 함수
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if step % 100 == 0: # 100번째 에포크마다 비용 출력
        print(step, cost.item())


0 0.7273974418640137
100 0.6931475400924683
200 0.6931471824645996
300 0.6931471824645996
400 0.6931471824645996
500 0.6931471824645996
600 0.6931471824645996
700 0.6931471824645996
800 0.6931471824645996
900 0.6931471824645996
1000 0.6931471824645996


200번 epochs 이후로 loss가 줄어들지 않음. 이는 단층 퍼셉트론으로 XOR 문제를 풀 수 없기 때문이다.

## 단층 퍼셉트론 예측

In [7]:
# 모델 평가, 예측 시 그래디언트 계산 비활성화. 가중치는 고정상태
# torch.no_grad(), detach() 중복하여 그래디언트 계산 중지
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    # detach() : 그래디언트 계산으로부터 분리
    print('모델의 출력값(Hypothesis): ', hypothesis.detach().cpu().numpy()) 
    print('모델의 예측값(Predicted): ', predicted.detach().cpu().numpy())
    print('실제값(Y): ', Y.cpu().numpy())
    print('정확도(Accuracy): ', accuracy.item())

모델의 출력값(Hypothesis):  [[0.5]
 [0.5]
 [0.5]
 [0.5]]
모델의 예측값(Predicted):  [[0.]
 [0.]
 [0.]
 [0.]]
실제값(Y):  [[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy):  0.5


In [9]:
# gpu 그대로 사용
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('모델의 출력값(Hypothesis): ', hypothesis)
    print('모델의 예측값(Predicted): ', predicted)
    print('실제값(Y): ', Y.cpu().numpy())
    print('정확도(Accuracy): ', accuracy.item())

모델의 출력값(Hypothesis):  tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], device='cuda:0')
모델의 예측값(Predicted):  tensor([[0.],
        [0.],
        [0.],
        [0.]], device='cuda:0')
실제값(Y):  [[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy):  0.5
